### Importations des packages

In [ ]:
from helpers.api_helpers import *
from helpers.other_helpers import *
pd.set_option('display.float_format', '{:.2f}'.format)


# Plan

0. Explication de ce notebook  
1. Extraction des données  
2. Préparation des données et calculs  

---

# 0. Ce qu'on veut faire sur ce notebook

Notre but, comme évoqué dans le **README**, est d'essayer d'expliquer la fréquentation des gares; mesurée en nombre de voyageurs en 2022, l'année la plus récente où nous avons toutes les données.  
Pour cela, on souhaite obtenir, pour chaque gare, les données suivantes :  

- **Type 1** :  
    - Données géographiques  
    - Fréquentations des gares issues du site [SNCF Open Data](https://ressources.data.sncf.com/pages/accueil/)  

- **Type 2** :  
    - Statistiques des données communales économiques issues du site [Une histoire du conflit politique](https://www.unehistoireduconflitpolitique.fr/telecharger.html)  

L'objectif principal est d'obtenir **le maximum de données** qui pourraient potentiellement expliquer la fréquentation des gares. Ensuite, grâce à des critères de sélection de variables dans notre modèle d'interprétation, nous analyserons les variables pertinentes pour l'explication.  

---

# 1. Extraction des données 

Dans cette section, nous allons :  
- Récupérer les données pertinentes via des API et des fichiers disponibles sur les sites web  
- Interagir avec **S3** pour stocker ou récupérer les données  

Commençons d'abord par les données du type 1


In [2]:
station_geo_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="gares-de-voyageurs",
    select="""nom as nom_gare,
              position_geographique,
              codeinsee,
              codes_uic as uic""",
)

station_freq_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="frequentation-gares",
    select="""total_voyageurs_2022,
              nom_gare,
              code_uic_complet as uic """,
)

nb of stations downloaded: 2881, from table gares-de-voyageurs
nb of stations downloaded: 3010, from table frequentation-gares


In [3]:
#fusion de ces 2 tables
station_geo_data = station_geo_data.rename(
    columns={
        "position_geographique.lon": "lon_gare",
        "position_geographique.lat": "lat_gare",
    }
)
station_geo_data = station_geo_data.drop("position_geographique", axis=1)

stations = station_freq_data.merge(
    station_geo_data.drop("nom_gare", axis=1), on=["uic"], how="left"
)
stations = stations[stations["total_voyageurs_2022"] > 0]#avoir les gares ouvertes


### **Maintenant, regardons comment importer des données de type 2**

Nous allons récupérer les **statistiques communales** nécessaires à notre analyse. Ces fichiers, disponibles initialement en format **zip/csv** sur le site indiqué plus haut, ont été **téléchargés et stockés manuellement sur S3** après décompression.

---

#### **Objectif**  
Importer ces données depuis le service de stockage **S3** pour les utiliser dans nos analyses.  

---

In [6]:
s3=s3_connection()

connection successful


On importe les données communales. Nous avons des fichiers de données sur le taux de personnes diplômées par commune, la population et le revenu par habitant.

1. Revenus par habitant et population par commune

In [7]:
columns_to_select = ["codecommune", "nomcommune", "pop2022", "revmoy2022"]
dtype_spec = {'codecommune': 'str'}

path_revcommunes = "clichere/diffusion/revcommunes.csv"
revcommunes = s3.read_csv_from_s3(path_revcommunes, columns_to_select, dtype_spec)

revcommunes.sample(2)


,codecommune,nomcommune,pop2022,revmoy2022
8074,23031,BOUSSAC,1198,14683.17
1955,06023,BREIL-SUR-ROYA,1880,18515.42


2. Taux de diplome par commune

In [8]:
columns_to_select = ["codecommune", "pbac2022", "psup2022"]
dtype_spec = {'codecommune': 'str'}

path_dipcommunes = "clichere/diffusion/diplomescommunes.csv"
dipcommunes = s3.read_csv_from_s3(path_dipcommunes, columns_to_select, dtype_spec)

dipcommunes.sample(2)


,codecommune,pbac2022,psup2022
4651,13076,0.44,0.29
12452,31145,0.53,0.37


3. Catégories socio-professionnelles

In [9]:
columns_to_select = ["codecommune","pagri2022", "pempl2022", "pcadr2022", "pouvr2022", "pchom2022", "pindp2022","ppint2022"]
dtype_spec = {'codecommune': 'str'}

path_cspcommunes = "clichere/diffusion/cspcommunes.csv"
cspcommunes = s3.read_csv_from_s3(path_cspcommunes, columns_to_select, dtype_spec)

cspcommunes.sample(2)

,codecommune,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
9706,25548,0.00,0.20,0.00,0.00,0.80,0.00,0.11
35669,85126,0.01,0.00,0.10,0.36,0.39,0.13,0.07


4. Fusion et export des fichiers de données

In [10]:
intermediaire_comm = pd.merge(revcommunes, dipcommunes, on='codecommune', how='inner')
#final_comm est une table finale pour les donées de type 2
final_comm = pd.merge(intermediaire_comm, cspcommunes, on="codecommune", how="inner")

final_comm.sample(2)

,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
8822,24566,VARENNES,496,15563.04,0.38,0.10,0.00,0.03,0.00,0.15,0.44,0.38,0.00
3428,10204,LONGEVILLE-SUR-MOGNE,196,16193.08,0.74,0.34,0.00,0.00,0.21,0.16,0.62,0.01,0.34


la table finale de type 2 est prête, donc il faut qu'elle puisse être utilisée par une autre personne qui ne va pas refaire ces étapes, donc il faut stocker sur s3


In [9]:
output_path1 = 'clichere/diffusion/final_comm.parquet'
output_path2 = 'clichere/diffusion/revcommunes.parquet'
output_path3 = 'clichere/diffusion/dipcommunes.parquet'
output_path4 = 'clichere/diffusion/cspcommunes.parquet'

s3.from_pandas_to_parquet_store_in_s3(final_comm, output_path1)
s3.from_pandas_to_parquet_store_in_s3(revcommunes, output_path2)
s3.from_pandas_to_parquet_store_in_s3(dipcommunes, output_path3)
s3.from_pandas_to_parquet_store_in_s3(cspcommunes, output_path4)


Cher lecteur, cette fonction écrit dans le dossier spécifié, mais vous n'avez pas les droits :( 
Cher lecteur, cette fonction écrit dans le dossier spécifié, mais vous n'avez pas les droits :( 
Cher lecteur, cette fonction écrit dans le dossier spécifié, mais vous n'avez pas les droits :( 
Cher lecteur, cette fonction écrit dans le dossier spécifié, mais vous n'avez pas les droits :( 


## 2. Préparation des données et calculs

Ici on va fusionner les 2 types de données et travailler sur les fusions, le nettoyage des données et les calculs supplémentaires, afin d'obtenir une table propre pour nos analyses.

In [11]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
regional_stat.sample(2)

,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
6847,19189,ST-BONNET-LES-TOURS,67,9512.22,0.51,0.07,0.00,0.00,0.00,0.78,0.00,0.22,0.47
35258,88208,GODONCOURT,182,7484.12,0.32,0.32,0.00,0.08,0.00,0.64,0.11,0.17,0.02


On supprime les communes qui n'ont pas d'habitants en 2022 ce qui compte pour 5% de la table.

Moins de 50 gares sont supprimés en utilisant le filtre sur le revenu moyen et prct du bac.

Ces informations peuvent manquer car certaines communes ont fusionné avant 2022 et donc la donnée n'est plus disponible pour ces communes.

In [12]:
old_nb_row=regional_stat.shape[0]
regional_stat = regional_stat[
    (regional_stat["pop2022"] > 0)
    & (regional_stat["revmoy2022"] > 0)
    & (regional_stat["pbac2022"].notna())
]
new_nb_row=regional_stat.shape[0]
print("prct communes supprimées",np.round((old_nb_row-new_nb_row)/old_nb_row,3)  )

prct communes supprimées 0.052


Ensuite, on fusionne les 2 types de données ensemble en se basant sur le code insee qui est la vraie clé de reconciliation. On s'assure que toutes les données sont assurées et qu'il y a plus de NA avec la commande *final.isna().sum()*

In [13]:
stations_region_stat=stations.merge(regional_stat,left_on="codeinsee",right_on="codecommune",how="left")
old_nb_row=stations_region_stat.shape[0]
#filtrer sur les communes non identifiés (car bases des données différentes), on a pas le moyen de les réidentifier 
stations_region_stat=stations_region_stat[stations_region_stat["pop2022"].notna()]
new_nb_row=stations_region_stat.shape[0]
print('prct gares supprimés', np.round((old_nb_row-new_nb_row)/old_nb_row,2) )
stations_region_stat.sample(2)

prct gares supprimés 0.04


,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
37,116030,Bartenheim,87182113,68021,7.49,47.63,68021,BARTENHEIM,3833.00,31280.21,0.48,0.26,0.00,0.07,0.18,0.18,0.35,0.22,0.07
441,54627,Saales,87214783,67421,7.11,48.35,67421,SAALES,802.00,13902.73,0.31,0.17,0.03,0.01,0.00,0.31,0.34,0.31,0.00


## Question soulevée

Une question se pose concernant l'analyse des données géographiques et communales disponibles.

### Données disponibles :

- Nous avons toutes les données **communales**, ce qui est très utile.
- Cependant, pour les **données géographiques**, nous ne disposons que des informations suivantes :
  - **Latitude** et **Longitude** des communes
  - **Nom de la commune**

### Problème :

Les informations géographiques (latitude, longitude) sont parfois insuffisantes pour une analyse plus poussée, car elles ne sont pas forcément très parlantes sans un contexte supplémentaire.

---

### Ce qui est important à considérer :

- **La population** des communes est bien sûr un facteur crucial.
- **La densité des gares** dans la région joue également un rôle significatif. En effet, une gare éloignée des autres gares pourrait potentiellement attirer un grand nombre de voyageurs de la région, ce qui peut influencer le transport et les flux de personnes.

### Proposition de solution :

- Une approche possible pour enrichir les données géographiques serait de calculer la **distance minimale** de chaque gare à son **voisin le plus proche**. 
- Cela permettrait d'obtenir une idée plus précise de l'accessibilité de chaque gare, ce qui pourrait être très utile pour certaines analyses.

Bien entendu, l'impact de la gare éloignée dépendra de nombreux facteurs, tels que la **taille de la ville**, la **commune**, ou même la **région** dans son ensemble. Mais, pour commencer, calculons déjà la distance minimale entre chaque gare et son voisin le plus proche.





In [14]:
#pour cela il faut faire cross join et on va obtenir pour chaque obsérvation toutes les obsérvations de la table ( table de nb de lignes  N*N)
cross_table_for_distance_calculation = stations_region_stat[["uic", "lon_gare", "lat_gare"]].merge(
    stations_region_stat[["uic", "lon_gare", "lat_gare"]], how="cross"
)
#nommer les variables 
(
    lat1,
    lon1,
    lat2,
    lon2,
) = (
    cross_table_for_distance_calculation["lat_gare_x"],
    cross_table_for_distance_calculation["lon_gare_x"],
    cross_table_for_distance_calculation["lat_gare_y"],
    cross_table_for_distance_calculation["lon_gare_y"],
)
#appliquer la formule haversine
cross_table_for_distance_calculation["dist_closest_station_km"] = haversine_vectorized(
    lat1, lon1, lat2, lon2
)
#enlever les gares par rapport à elles-mêmes ( N obsérvations)
cross_table_for_distance_calculation = cross_table_for_distance_calculation[
    cross_table_for_distance_calculation["uic_x"]
    != cross_table_for_distance_calculation["uic_y"]
]
#grouper par les gares et sortir les indices de distance minimale
idx = cross_table_for_distance_calculation.groupby(["uic_x"])[
    "dist_closest_station_km"
].idxmin()
#sortir toutes les infos sur ces gares
result_dist = cross_table_for_distance_calculation.loc[idx].reset_index(drop=True)
#merger pour avoir l'ensemble de l'info
stations_region_stat = stations_region_stat.merge(
    result_dist[["uic_x", "dist_closest_station_km"]].rename(columns={"uic_x": "uic"}),
    on="uic",
    how="inner",
)
stations_region_stat.sample(2)

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km
2508,173657,Lillers,87342238,62516,2.48,50.56,62516,LILLERS,9751.00,13556.27,0.40,0.21,0.00,0.07,0.06,0.27,0.34,0.25,0.09,3.29
1200,11089,Barenton-Bugny,87297291,02046,3.66,49.63,02046,BARENTON-BUGNY,517.00,17696.86,0.39,0.20,0.04,0.01,0.00,0.40,0.15,0.40,0.00,2.16


Pour la densité des gares par région, il serait juste intéressant de regarder combien de gares il y a dans la même commune, donc sortons cette information.

In [15]:

communes_count=stations_region_stat.groupby('nomcommune').agg({'nomcommune': ['count']})
communes_count["commune"]=communes_count.index
communes_count=communes_count.reset_index(drop=True)
communes_count.columns = ["nb_stations_same_commune","nomcommune"]
stations_region_stat=stations_region_stat.merge(communes_count,on="nomcommune",how="inner")


La **commune** peut être considérée comme une petite unité, similaire à une **observation** dans notre analyse. En effet, la majorité des gares se trouvent dans une seule commune, comme l'indique la variable **nb_stations_same_communes**.

### Objectif :

Pour réaliser une **visualisation** ou un **modèle**, il est nécessaire de regrouper les communes en **régions**. Nous allons nous appuyer sur les données géographiques disponibles pour effectuer ce regroupement.

### Informations disponibles pour chaque commune :

1. **Latitude** et **Longitude** (données géographiques)
2. **Nom de la commune** (comme dernier recours, si la latitude et longitude sont introuvables)

---

### Méthodologie proposée :

1. **Utiliser les données géographiques** (latitude et longitude) pour identifier la région associée à chaque commune.
   
   - Nous allons envoyer ces informations à l'**API gouvernementale** (Gouv.fr) qui peut retourner la région correspondante.

2. **Cas où les coordonnées géographiques sont manquantes** :
   
   - Si les informations géographiques (latitude, longitude) sont introuvables ou incorrectes, nous utiliserons le **nom de la commune** comme dernier recours pour déterminer la région.

---

### Variables à générer :

- **Région** : Une variable catégorielle qui devrait discriminer le plus possible la **charge dans les gares**. Cette région sera utilisée pour analyser la densité de gares et l'accessibilité dans chaque zone.
   


In [16]:
#générer la variable region comme expliqué 
french_regions=gouv_api_addresses(stations_region_stat)

In [17]:
stations_region_stat["regions"]=french_regions
#voilà les résultats de l'api,tout est bien renseigné
print(stations_region_stat["regions"].unique())
stations_region_stat.sample(2)


['Pays de la Loire' 'Île-de-France' 'Hauts-de-France'
 "Provence-Alpes-Côte d'Azur" 'Occitanie' 'Auvergne-Rhône-Alpes'
 'Bourgogne-Franche-Comté' 'Centre-Val de Loire' 'Normandie'
 'Nouvelle-Aquitaine' 'Grand Est' 'Bretagne']


,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,...,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km,nb_stations_same_commune,regions
2800,48557,Voves,87545707,28422,1.62,48.28,28422,VOVES,5214.00,12081.20,...,0.02,0.06,0.02,0.28,0.30,0.32,0.03,20.08,1,Centre-Val de Loire
1227,5302890,Bois-Colombes,87381079,92009,2.27,48.91,92009,BOIS-COLOMBES,27599.00,32633.94,...,0.00,0.05,0.49,0.25,0.14,0.07,0.04,0.89,2,Île-de-France


Tout est bon! On a les données géographiques et économiques, exportons sur S3 et vérifions la connection


In [ ]:
s3.from_pandas_to_parquet_store_in_s3(stations_region_stat,"aayrapetyan/diffusion/final_table.parquet")

In [ ]:
stations_region_stat=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
stations_region_stat

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,...,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km,regions
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,...,0.17,0.06,0.02,0.05,0.14,0.31,0.42,0.04,9.75,Pays de la Loire
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,...,0.64,0.00,0.04,0.39,0.27,0.23,0.07,0.06,1.97,Île-de-France
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,...,0.34,0.00,0.00,0.11,0.35,0.16,0.38,0.08,4.22,Hauts-de-France
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,...,0.34,0.00,0.13,0.13,0.25,0.35,0.13,0.10,1.80,Provence-Alpes-Côte d'Azur
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,...,0.20,0.01,0.14,0.08,0.21,0.39,0.17,0.11,5.56,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,...,0.26,0.01,0.15,0.23,0.33,0.22,0.07,0.03,2.10,Grand Est
2804,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,...,0.13,0.00,0.09,0.09,0.23,0.24,0.34,0.04,1.82,Grand Est
2805,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,...,0.28,0.01,0.07,0.15,0.27,0.21,0.30,0.06,4.08,Hauts-de-France
2806,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,...,0.28,0.00,0.08,0.11,0.17,0.33,0.32,0.09,6.68,Nouvelle-Aquitaine
